In [ ]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import numpy as np
import traceback
from keras.models import load_model
from string import ascii_uppercase
import mediapipe as mp


class Application:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Sign Language To Text Conversion")
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        self.root.geometry("1300x700")

        # Initialize variables
        self.vs = cv2.VideoCapture(0)
        self.current_image = None
        self.current_image2 = None
        self.ccc = 0
        self.current_symbol = ""
        self.str = ""
        self.blank_flag = 0
        self.ct = {char: 0 for char in ascii_uppercase}
        self.prev_char = ''
        self.count = 0

        # Load model
        print("Loading model...")
        self.model = load_model(r'A:\desktop\progit\Sign-Language-To-Text-and-Speech-Conversion\cnn8grps_rad1_model.h5')
        print("Model loaded successfully.")

        self.setup_gui()
        self.video_loop()

    def setup_gui(self):
        self.panel = tk.Label(self.root)
        self.panel.place(x=10, y=10, width=640, height=480)

        self.panel2 = tk.Label(self.root)
        self.panel2.place(x=660, y=10, width=640, height=480)

        self.panel3 = tk.Label(self.root)
        self.panel3.place(x=660, y=500)

        self.panel5 = tk.Label(self.root)
        self.panel5.place(x=10, y=500, width=640, height=200)

    def destructor(self):
        print("Closing Application")
        self.root.destroy()
        self.vs.release()
        cv2.destroyAllWindows()

    def video_loop(self):
        mp_hands = mp.solutions.hands
        hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
        mp_drawing = mp.solutions.drawing_utils

        try:
            ret, frame = self.vs.read()
            if not ret:
                print("Failed to grab frame")
                self.root.after(10, self.video_loop)
                return

            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = hands.process(frame_rgb)

            if result.multi_hand_landmarks:
                for hand_landmarks in result.multi_hand_landmarks:
                    x_min, y_min = float('inf'), float('inf')
                    x_max, y_max = float('-inf'), float('-inf')

                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        x_min, y_min = min(x_min, x), min(y_min, y)
                        x_max, y_max = max(x_max, x), max(y_max, y)

                    bbox = (x_min, y_min, x_max - x_min, y_max - y_min)
                    image = frame[y_min:y_max, x_min:x_max]
                    white = np.ones((400, 400, 3), np.uint8) * 255

                    h, w, _ = image.shape
                    os_x = (400 - w) // 2
                    os_y = (400 - h) // 2

                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        cv2.circle(white, (x - x_min + os_x, y - y_min + os_y), 5, (0, 0, 255), -1)

                    mp_drawing.draw_landmarks(white, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    white_resized = cv2.resize(white, (400, 400))
                    white_expanded = np.expand_dims(white_resized, axis=0)
                    white_expanded = np.array(white_expanded, dtype='float32')

                    prob = np.array(self.model.predict(white_expanded)[0], dtype='float32')
                    ch1 = np.argmax(prob, axis=0)
                    prob[ch1] = 0
                    ch2 = np.argmax(prob, axis=0)
                    prob[ch2] = 0
                    ch3 = np.argmax(prob, axis=0)
                    prob[ch3] = 0

                    print(f"Predicted classes: {ch1}, {ch2}, {ch3}")

                    # Update GUI with the new image
                    white = cv2.cvtColor(white, cv2.COLOR_BGR2RGB)
                    img = Image.fromarray(white)
                    imgtk = ImageTk.PhotoImage(image=img)
                    self.panel2.imgtk = imgtk
                    self.panel2.config(image=imgtk)

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            self.panel.imgtk = imgtk
            self.panel.config(image=imgtk)

        except Exception as e:
            print(f"An error occurred: {e}")
            traceback.print_exc()

        self.root.after(10, self.video_loop)

    def draw_hand_lines(self, white, os_x, os_y, pts):
        for t in range(0, 4):
            cv2.line(white, (pts[t][0] + os_x, pts[t][1] + os_y), 
                     (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), (0, 255, 0), 3)
        for t in range(5, 8):
            cv2.line(white, (pts[t][0] + os_x, pts[t][1] + os_y), 
                     (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), (0, 255, 0), 3)
        for t in range(9, 12):
            cv2.line(white, (pts[t][0] + os_x, pts[t][1] + os_y), 
                     (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), (0, 255, 0), 3)
        for t in range(13, 16):
            cv2.line(white, (pts[t][0] + os_x, pts[t][1] + os_y), 
                     (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), (0, 255, 0), 3)
        for t in range(17, 20):
            cv2.line(white, (pts[t][0] + os_x, pts[t][1] + os_y), 
                     (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), (0, 255, 0), 3)
        for t in [0, 5, 9, 13, 17]:
            cv2.line(white, (pts[t][0] + os_x, pts[t][1] + os_y), 
                     (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), (0, 255, 0), 3)
            cv2.line(white, (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), 
                     (pts[t + 2][0] + os_x, pts[t + 2][1] + os_y), (0, 255, 0), 3)
        for t in range(1, 20, 4):
            cv2.line(white, (pts[0][0] + os_x, pts[0][1] + os_y), 
                     (pts[t][0] + os_x, pts[t][1] + os_y), (0, 255, 0), 3)


if __name__ == "__main__":
    print("Starting Application...")
    app = Application()
    app.root.mainloop()


Starting Application...
Loading model...
Model loaded successfully.
